In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from trl import SFTTrainer
import os,torch, wandb, platform, gradio, warnings
from pyvi import ViTokenizer


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import transformers
torch.__version__, transformers.__version__

('2.7.0+cu118', '4.51.3')

In [4]:
torch.cuda.is_available()

True

### load dataset

In [5]:
filename_train = "data_pseudo_vireport/train_data_high.csv"
filename_test = "data_pseudo_vireport/final_test.csv"

data_train = pd.read_csv(filename_train)
data_test = pd.read_csv(filename_test)
data_train

,Sentences,labels
0,Thêm dấu phẩy sau trên 700 nghìn tỷ đồng vốn đ...,0
1,Lựa chọn cổ phiếu niêm yết có chỉ số PB cao đư...,0
2,Chính sách bảo hiểm nông nghiệp là tia hy vọng...,2
3,Ngoài ra Vingroup cam kết phát triển các dự án...,2
4,Diện tích trồng cà gai leo tại địa phương là k...,1
...,...,...
37631,Chúng tôi là một trong những công ty Thụy Sĩ đ...,1
37632,Chúng ta sẽ đạt được mục tiêu này bằng cách tr...,1
37633,Điều này có nghĩa là cải thiện hiệu quả điện n...,1
37634,Năm 2021 chúng tôi đã tiến hành bước đầu tiên ...,1


In [6]:
data_train['labels'].value_counts()

labels
2    14155
0     9511
3     8203
1     5767
Name: count, dtype: int64

In [7]:
data_test['labels'].value_counts()

labels
2    996
3    924
1    570
0    510
Name: count, dtype: int64

### load FinBERT pretrained model
The pretrained FinBERT model path on Huggingface is https://huggingface.co/yiyanghkust/finbert-pretrain

In [8]:
import torch
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# for param in model.bert.parameters():
#     param.requires_grad = False

# Keep only the classification head trainable
for param in model.classifier.parameters():
    param.requires_grad = True

### prepare dataset for fine-tuning

In [10]:
from datasets import Dataset
from gensim.utils import simple_preprocess

def preprocess_text(text):
    tokens = simple_preprocess(text)
    # Ghép lại thành câu
    text = ' '.join(tokens)
    # Tách từ kiểu tiếng Việt (giữ định dạng từ ghép có dấu gạch dưới)
    text = ViTokenizer.tokenize(text)
    return text

# Tiền xử lý cột văn bản trong từng DataFrame
data_train['Sentences'] = data_train['Sentences'].apply(preprocess_text)
data_test['Sentences'] = data_test['Sentences'].apply(preprocess_text)

# Chuyển sang Dataset của HuggingFace
dataset_train = Dataset.from_pandas(data_train)
dataset_val = Dataset.from_pandas(data_train)


dataset_train = dataset_train.map(lambda e: tokenizer(e['Sentences'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['Sentences'], truncation=True, padding='max_length', max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/37636 [00:00<?, ? examples/s]

Map:   0%|          | 0/37636 [00:00<?, ? examples/s]

In [11]:
dataset_train

Dataset({
    features: ['Sentences', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 37636
})

In [12]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
data_collator = DataCollatorWithPadding(tokenizer)

# Tạo DataLoader
train_dataloader = DataLoader(
    dataset_train, 
    batch_size=32, 
    collate_fn=data_collator,
    shuffle=True
)

# Kiểm tra kích thước batch
for batch in train_dataloader:
    print("Input IDs shape:", batch["input_ids"].shape)
    print("Labels shape:", batch["labels"].shape)
    print((batch["labels"][0]))
    break


Input IDs shape: torch.Size([32, 128])
Labels shape: torch.Size([32])
tensor(0)


### define training options

In [13]:
wandb.login(key="e7056d3a3bc855a6e0d38b8c4ff7d2ec7ce895af")
run = wandb.init(project='bert-base', job_type="training", anonymous="allow")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pc/.netrc
wandb: Currently logged in as: trung235689 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:
print(model.config.num_labels)  


4


In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        eval_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate= 2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,  # 1 epoch
        weight_decay=0.005,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)


trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()   

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.585500,0.426123,0.842996
2,0.411600,0.332087,0.872622
3,0.262400,0.166827,0.940669
4,0.191000,0.089364,0.971782
5,0.133600,0.062315,0.981799


TrainOutput(global_step=5885, training_loss=0.34505601168286526, metrics={'train_runtime': 1508.0819, 'train_samples_per_second': 124.781, 'train_steps_per_second': 3.902, 'total_flos': 1.237828187461632e+16, 'train_loss': 0.34505601168286526, 'epoch': 5.0})












### evaluate on testing set

In [16]:
model.eval()
trainer.predict(dataset_val).metrics

{'test_loss': 0.0623147152364254,
 'test_accuracy': 0.9817993410564353,
 'test_runtime': 66.5771,
 'test_samples_per_second': 565.3,
 'test_steps_per_second': 17.679}

### save the fine-tuned model

In [17]:
trainer.save_model('bert-base-train/')
tokenizer.save_pretrained("./bert-base-train/")


('./bert-base-train/tokenizer_config.json',
 './bert-base-train/special_tokens_map.json',
 './bert-base-train/vocab.txt',
 './bert-base-train/added_tokens.json')

In [19]:
from sklearn.metrics import accuracy_score

# Dự đoán
model.eval()
predictions = trainer.predict(dataset_val)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

overall_accuracy = accuracy_score(labels, preds)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Overall Accuracy: 0.9818


In [18]:
model.eval()
predictions = trainer.predict(dataset_val)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# --- 9. Hàm tính accuracy theo từng lớp ---
def save_per_class_accuracy(y_true, y_pred, class_names=None, output_path="bert-base_per_class_accuracy.csv"):
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.cpu().numpy()

    labels = np.unique(y_true)
    acc_per_class = []

    for label in labels:
        idx = y_true == label
        acc = accuracy_score(y_true[idx], y_pred[idx])
        class_label = class_names[label] if class_names else f"Class {label}"
        acc_per_class.append((class_label, acc))

    df = pd.DataFrame(acc_per_class, columns=["Class", "Accuracy"])
    print(df)
    df.to_csv(output_path, index=False)

# --- 10. Gọi hàm ---
class_names = ["0", "1", "2", "3"]  # bạn có thể thay bằng tên thật của từng lớp nếu có
save_per_class_accuracy(labels, preds, class_names)

  Class  Accuracy
0     0  0.965724
1     1  0.989769
2     2  0.989898
3     3  0.980861
